<a href="https://colab.research.google.com/github/gabrielubiale/IA-e-NLP---analise-sobre-fake-news/blob/main/FakeNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import tensorflow as tf

In [ ]:
df = pd.read_csv("allBase.csv")
df.head()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Criar o gráfico de contagem
sns.countplot(x="category", data=df)

# Configurar os rótulos dos eixos
plt.xlabel("0 = true 1 = fake")
plt.ylabel("Contagem")

# Exibir o gráfico
plt.show()


Stop words são palavras comuns e frequentes em um idioma que geralmente são removidas durante o processamento de texto, como em tarefas de análise de texto, processamento de linguagem natural e recuperação de informações. Essas palavras não fornecem informações distintivas sobre o conteúdo do texto e podem ser consideradas "ruído" em muitos casos.

As stop words geralmente incluem pronomes, preposições, conjunções e outras palavras muito comuns que não contribuem significativamente para o significado geral de um texto. Alguns exemplos de stop words em inglês são "a", "an", "the", "in", "on", "is", "and", "but", entre outras.

A remoção de stop words é uma etapa comum no pré-processamento de texto para reduzir a dimensionalidade do texto e melhorar a eficiência e a qualidade de análises subsequentes, como análise de sentimentos, classificação de textos e modelagem de tópicos. A ideia por trás da remoção de stop words é se concentrar nas palavras-chave mais significativas e relevantes em um texto, que podem fornecer insights valiosos para a análise ou modelagem em questão.

In [ ]:
import nltk
from nltk.corpus import stopwords
import string

# Baixar os stopwords em inglês
nltk.download('stopwords')

# Criar um conjunto de stopwords em inglês
stop = set(stopwords.words('english'))

# Criar uma lista de pontuações
# é uma string contendo todos os caracteres de pontuação.
punctuation = list(string.punctuation)

# Adicionar as pontuações ao conjunto de stopwords
stop.update(punctuation)
print(stop)


In [ ]:
def strip_html(text):
    """
    Remove tags HTML do texto utilizando a biblioteca BeautifulSoup.
    """
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    """
    Remove o conteúdo entre colchetes quadrados do texto utilizando expressões regulares.
    """
    return re.sub('\[[^]]*\]', '', text)

def remove_urls(text):
    """
    Remove URLs do texto utilizando expressões regulares.
    """
    return re.sub(r'http\S+', '', text)

def remove_stopwords(text):
    """
    Remove as stopwords do texto.
    """
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)

def denoise_text(text):
    """
    Aplica uma sequência de etapas de pré-processamento para remover ruídos do texto.
    """
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text

# Aplica as funções na coluna text
df['text'] = df['text'].apply(denoise_text)


#MELHORIAS:
#rodar todo o codigo novamente
#limpeza de dados

Até esse momento houveram algumas configurações e normalização

In [ ]:
#cria uma nuvem de palavras com o novo data set para os dados fake
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

# Define o tamanho da figura
plt.figure(figsize=(20, 20))

# Cria a nuvem de palavras
wc = WordCloud(max_words=10000, width=1600, height=800, stopwords=STOPWORDS).generate(" ".join(df[df.category == 1].text))

# Exibe a imagem da nuvem de palavras
plt.imshow(wc, interpolation='bilinear')

# Mostra o gráfico
plt.show()

In [ ]:
#cria uma nuvem de palavras com o novo data set para os dados true

import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

plt.figure(figsize=(20, 20))  # Define o tamanho da figura do gráfico

# Cria a nuvem de palavras
wc = WordCloud(max_words=2000, width=1600, height=800, stopwords=STOPWORDS).generate(" ".join(df[df.category == 0].text))

plt.imshow(wc, interpolation='bilinear')  # Exibe a imagem da nuvem de palavras

plt.show()  # Exibe o gráfico

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))
# Cria uma figura com dois subplots lado a lado, com uma largura de 12 unidades e altura de 8 unidades.

text_len = df[df['category'] == 1]['text'].str.len()
ax1.hist(text_len, color='red')
# Obtém o comprimento dos textos na coluna 'text' onde a categoria é igual a 1.
# Plota um histograma dos comprimentos dos textos no primeiro subplot (ax1), usando a cor vermelha.

ax1.set_title('Comprimento dos textos fake')
# Define o título do primeiro subplot como 'Original text'.

text_len = df[df['category'] == 0]['text'].str.len()
ax2.hist(text_len, color='green')
# Obtém o comprimento dos textos na coluna 'text' onde a categoria é igual a 0.
# Plota um histograma dos comprimentos dos textos no segundo subplot (ax2), usando a cor verde.

ax2.set_title('Comprimento dos textos verdadeiros')
# Define o título do segundo subplot como 'Fake text'.

fig.suptitle('Número de caracteres no texto')
# Define o título da figura principal como 'Characters in texts'.

plt.show()
# Exibe a figura com os subplots.


#esses subplots mostram a distribuição dos comprimentos dos textos nas categorias 0 e 1.
#Compara-se os dois plots para ver diferença de comprimento
#entre os textos verdadeiros e falsos

In [ ]:
# Criar uma figura com dois subplots lado a lado
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))

# Calcular o número de palavras nos textos da categoria 1 (verdadeiros) e criar um histograma
text_len = df[df['category'] == 1]['text'].str.split().map(lambda x: len(x))
ax1.hist(text_len, color='red')
ax1.set_title('Texto falso')

# Calcular o número de palavras nos textos da categoria 0 (falsos) e criar um histograma
text_len = df[df['category'] == 0]['text'].str.split().map(lambda x: len(x))
ax2.hist(text_len, color='green')
ax2.set_title('Texto verdadeiro')

# título do gráfico
fig.suptitle('Comparação entre o numero de palavras dos textos falsos e verdadeiros')

# Exibir a figura
plt.show()

# mostram a distribuição do número de palavras nos textos

In [ ]:
# Criação da figura e dos dois subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

# Análise dos textos originais (categoria 1)
word = df[df['category'] == 1]['text'].str.split().apply(lambda x: [len(i) for i in x])
sns.distplot(word.map(lambda x: np.mean(x)), ax=ax1, color='red')
ax1.set_title('dados falsos')

# Análise dos textos falsos (categoria 0)
word = df[df['category'] == 0]['text'].str.split().apply(lambda x: [len(i) for i in x])
sns.distplot(word.map(lambda x: np.mean(x)), ax=ax2, color='green')
ax2.set_title('dados verdadeiros')

# Configuração do título principal da figura
fig.suptitle('Comprimento médio das palavras por categoria')

#Mostram a distribuição do comprimento médio das palavras nos textos das categorias
#tamanho médio das palavras por texto


#MELHORIA:

#analisar melhor o gráfico e fazer a curva padrão

In [ ]:
def get_corpus(text):
    # Cria uma lista vazia para armazenar as palavras
    words = []

    # Percorre cada texto no parâmetro 'text'
    for i in text:
        # Divide o texto em palavras usando o método split()
        for j in i.split():
            # Remove espaços em branco no início e no final de cada palavra e adiciona à lista 'words'
            words.append(j.strip())

    # Retorna a lista de palavras
    return words

# Chama a função 'get_corpus' passando o texto do DataFrame 'df' como argumento e armazena o resultado na variável 'corpus'
corpus = get_corpus(df.text)

#corpus é uma lista que começa no zero e vai até o 20° elemento
corpus[:21]

#"corpus" se refere a uma lista de palavras extraídas dos textos presentes na coluna 'text' do DataFrame df.
#A função get_corpus é usada para percorrer cada texto dividir em palavras usando o método split(), remover
#espaços em branco em cada palavra e adicioná-las na lista
#Corpus é uma lista que contém todas as palavras presentes nos textos.
#fornece uma coleção de palavras que podem ser utilizadas para diversas finalidades

In [ ]:
def get_corpus(text):
    words = []

    for i in text:
        for j in i.split():
            words.append(j.strip())
    return words

category_1_text = df[df['category'] == 1]['text']

corpus = get_corpus(category_1_text)

corpus[:20] #corpus é uma lista que começa no zero e vai até o 20° elemento


In [ ]:
from collections import Counter

# Cria um objeto Counter para contar a ocorrência de cada elemento no corpus
counter = Counter(corpus)

# Obtém as 10 palavras mais comuns do corpus
most_common = counter.most_common(20)

# Converte a lista de tuplas em um dicionário para facilitar o acesso aos dados
most_common = dict(most_common)

# Imprime o dicionário com as palavras mais comuns e suas contagens
most_common


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def get_top_text_ngrams(corpus, n, g):
    """
    Retorna os n-gramas mais frequentes em um corpus de texto.

    Parâmetros:
        - corpus: lista de documentos de texto.
        - n: número de n-gramas mais frequentes a serem retornados.
        - g: tamanho dos n-gramas (1 para unigramas, 2 para bigramas, etc.).

    Retorna:
        - Lista de tuplas contendo o n-grama e sua contagem de ocorrências, ordenada por frequência decrescente.
    """

    # Inicializa o objeto CountVectorizer com o tamanho dos n-gramas
    vec = CountVectorizer(ngram_range=(g, g)).fit(corpus)

    # Transforma o corpus em uma matriz esparsa de contagens de n-gramas
    bag_of_words = vec.transform(corpus)

    # Calcula a soma das contagens de cada n-grama
    sum_words = bag_of_words.sum(axis=0)

    # Cria uma lista de tuplas contendo o n-grama e sua contagem
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]

    # Ordena a lista de tuplas por frequência decrescente
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)

    # Retorna os n-gramas mais frequentes
    return words_freq[:n]


#A função retorna uma lista de tuplas contendo os n-gramas mais frequentes, ordenados por frequência decrescente
#N-gramas são sequências de palavras em um texto. Eles nos ajudam a entender a estrutura e o significado desse texto.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Define o tamanho da figura do gráfico
plt.figure(figsize=(16, 9))

# Obtém os unigramas mais comuns a partir da coluna 'text' do DataFrame 'df'
most_common_uni = get_top_text_ngrams(df.text, 20, 1)

# Converte o resultado em um dicionário
most_common_uni = dict(most_common_uni)

# Cria um gráfico de barras utilizando o seaborn
# O eixo x contém os valores dos unigramas mais comuns (contagem) e o eixo y contém as chaves (palavras)
sns.barplot(x=list(most_common_uni.values()), y=list(most_common_uni.keys()))

#visualizar os n-gramas mais comuns em um texto.
#visualizar a sequencia de palavras mais comuns

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Define o tamanho da figura do gráfico
plt.figure(figsize=(16, 9))

# Obtém os 10 bigramas mais comuns no texto
most_common_bi = get_top_text_ngrams(df[df['category'] == 1]['text'], 20, 2)

# Converte o resultado para um dicionário
most_common_bi = dict(most_common_bi)

# Cria um gráfico de barras com os bigramas mais comuns
sns.barplot(x=list(most_common_bi.values()), y=list(most_common_bi.keys()))

# Exibe o gráfico
plt.show()

#gráfico de barras que mostra os bigramas mais frequentes em um texto,
#permitindo uma visualização combinações de duas palavras mais comuns

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Definir o tamanho da figura
plt.figure(figsize=(16, 9))

# Obter os top 10 trigramas mais comuns do texto
most_common_tri = get_top_text_ngrams(df.text, 20, 3)

# Converter o resultado em um dicionário
most_common_tri = dict(most_common_tri)

# Criar o gráfico de barras utilizando o Seaborn
sns.barplot(x=list(most_common_tri.values()), y=list(most_common_tri.keys()))

# Exibir o gráfico
plt.show()

#trigramas